# 여기서부터

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow import nn
import pandas as pd
import numpy as np
import pickle
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2023-04-11 22:43:35.131987: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import pandas as pd


import pickle
# train
with open("augmentedhalf1.pkl","rb") as fr:
    data = pickle.load(fr)


In [3]:
with open("valid.pkl","rb") as fr:
    valid = pickle.load(fr)
valid=valid.drop(['Segment ID'], axis=1)

In [4]:

x_train, y_train = data[['temp+eda','text_tokenize','mfcc_scaled']], data['sentiment']
x_valid, y_valid = valid[['temp+eda','text_tokenize','mfcc_scaled']], valid['sentiment_x']

# audio, text 임베딩

https://wikidocs.net/103802  참고

In [5]:
import tensorflow as tf
class a_TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super().__init__()
        self.a_emb = tf.keras.layers.Conv1D(filters=32, kernel_size=551,padding='valid', activation='relu',input_shape=(600,50)) #(1,50,32)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim) #(50,32)

    def call(self, x):
        maxlen = 50
        
        positions = tf.range(start=0, limit=maxlen, delta=1)

        positions = self.pos_emb(positions)

        x = self.a_emb(tf.convert_to_tensor(x)) #self.a_emb(np.array([x]))[0] #오디오 임베딩
        
        return x + positions



class t_TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen,vocab_size, embed_dim):
        super().__init__()
        self.t_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim) #텍스트 임베딩(50*300을 50*1 로 바꿔줘야하는데, 그냥 okt, 케라스 토크나이저 사용해서 쪼개자 그냥 (maxlen=50으로))
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen=50
        print("t")
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)

        x = self.t_emb(x) #텍스트 임베딩
        return x + positions


# a->t 트랜스포머 (쿼리가 텍스트, 키 밸류는 오디오), 타겟:텍스트, 소스: 오디오

텍스트, 오디오 순서 인풋

In [6]:
class at_MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(at_MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs_text, inputs_audio): ##################################################33
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs_text)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs_text)
        key = self.key_dense(inputs_audio)
        value = self.value_dense(inputs_audio)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs
    
class at_co_TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = at_MultiHeadAttention(embedding_dim=32, num_heads=4)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs_text, inputs_audio, training):
        attn_output = self.att(inputs_text, inputs_audio)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs_text + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# ta_트랜스포머  : 오디오 텍스트 순서 input

In [7]:

class ta_MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(ta_MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs_audio, inputs_text): ##################################################33
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs_audio)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs_audio)
        key = self.key_dense(inputs_text)
        value = self.value_dense(inputs_text)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs
    
class ta_co_TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = ta_MultiHeadAttention(embedding_dim=32, num_heads=4)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs_audio, inputs_text, training):
        attn_output = self.att(inputs_audio, inputs_text)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs_audio + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        print("text->au 트랜스포머")
        return self.layernorm2(out1 + ffn_output)

# bio 트랜스포머
#바이오는 co-attetnion이 아니고 그냥 concat해서 input되기때문에 그냥 트랜스포머 block을 사용

In [8]:
class bio_TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super().__init__()

        self.bio_temp_emb = layers.Embedding(input_dim=80, output_dim=embed_dim) #토큰 
        self.bio_eda_emb = layers.Embedding(input_dim=80, output_dim=embed_dim) #토큰 임베딩
        print(self.bio_temp_emb)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = 160
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)

        y1 = self.bio_temp_emb(tf.slice(x,[0,0],[-1,80])) #토큰임베딩 진행
        y2= self.bio_eda_emb(tf.slice(x,[0,80],[-1,-1]))

        x=tf.concat([y1,y2],axis=1)
        

        return x + positions

class TransformerBlock(layers.Layer):  #공통적인 트랜스포머 블럭
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = MultiHeadAttention(embedding_dim=embed_dim, num_heads=num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        #print('어텐션 후',attn_output.shape)
        attn_output = self.dropout1(attn_output, training=training)
        #print('드롭아웃',attn_output.shape)
        out1 = self.layernorm1(inputs + attn_output)
        #print('레이어놈',out1.shape)
        ffn_output = self.ffn(out1)
        #print('ffn후',ffn_output.shape)
        ffn_output = self.dropout2(ffn_output, training=training)
        print("트랜스포머")
        return self.layernorm2(out1 + ffn_output)

In [9]:
class MultiHeadAttention(tf.keras.layers.Layer): #공통적인 트랜스포머 멀티헤드어텐션
    def __init__(self, embedding_dim, num_heads=4):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs

In [10]:

y_valid = to_categorical(y_valid)
y_train = to_categorical(y_train)


In [11]:

from tensorflow.keras import backend as K
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score


In [12]:
embed_dim = 32 # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen=160
training=1

In [22]:

audio_input = layers.Input(shape=(600,50)) #
text_input = layers.Input(shape=(50,)) #
bio_input = layers.Input(shape=(160,)) #

    #input_audio text bio 는 따로 정해줘야함

a_embedding_layer = a_TokenAndPositionEmbedding(600, 32) 
print("audio",audio_input.shape)
audio_embedding = a_embedding_layer(audio_input)######### ################################################################3

t_embedding_layer = t_TokenAndPositionEmbedding(50,14991, embed_dim)
text_embedding = t_embedding_layer(text_input)  ########## 

b_embedding_layer = bio_TokenAndPositionEmbedding(160, embed_dim)
bio_embedding = b_embedding_layer(tf.convert_to_tensor(bio_input))  ########## 




    # 멀티모달 co-attention
at_transformer_block = at_co_TransformerBlock(32, 4, 32)
ta_transformer_block = ta_co_TransformerBlock(embed_dim, num_heads, ff_dim)
bio_transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
transformer_block = TransformerBlock(96, num_heads, ff_dim)

    #진행되는 부분
output_at = at_transformer_block(text_embedding, audio_embedding) ############ (50,50,32)
output_ta = ta_transformer_block(audio_embedding, text_embedding) # (50,50,32)
output_bio = bio_transformer_block(bio_embedding) # (160,160,32)

output_at = layers.GlobalAveragePooling1D()(output_at)
output_ta = layers.GlobalAveragePooling1D()(output_ta)
output_bio = layers.GlobalAveragePooling1D()(output_bio)

concat_embedding = layers.concatenate([output_at, output_ta, output_bio])#tf.concat([output_at, output_ta, output_bio], axis=0) #concat 진행

output = transformer_block(concat_embedding) 
output = layers.Dense(100, activation="sigmoid")(output)

output = transformer_block(output)############ concat 후 트랜스포머 블럭


x = layers.GlobalAveragePooling1D()(output)

x = layers.Dropout(0.2)(x)
x = layers.Dense(50, activation="sigmoid")(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(20, activation="sigmoid")(x)
x = layers.Dropout(0.2)(x)

outputs = layers.Dense(7, activation="softmax")(x)  #클래시파이어(분류기)
print(outputs.shape)

model = keras.Model(inputs=[audio_input, text_input, bio_input], outputs=outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy', precision, recall, f1score])
model.summary()
    


    #멀티인풋->하나의 아웃풋이어야함
    #트랜스포머 블럭별로 하나의 Model 만들어서 하기

audio (None, 600, 50)
t
before trans (None, 50, 32) (None, 50, 32) (None, 160, 32)
text->au 트랜스포머
트랜스포머
after trans (None, 50, 32) (None, 50, 32) (None, 160, 32)
after pooling (None, 32) (None, 32) (None, 32)
concat (None, 96)
트랜스포머
output (None, None, 100)
(None, 7)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 600, 50)]    0                                            
______________________

In [55]:
history = model.fit(
        [[np.array(x_train['mfcc_scaled'].to_list()),np.array(x_train['text_tokenize'].to_list()),np.array(x_train['temp+eda'].to_list())]], y_train, batch_size=16, epochs=5, validation_data=([[np.array(x_valid['mfcc_scaled'].to_list()),np.array(x_valid['text_tokenize'].to_list()),np.array(x_valid['temp+eda'].to_list())]], y_valid)
    )
#model.save('model.h5')



Epoch 1/5
t
트랜스포머
text->au 트랜스포머
트랜스포머
t
트랜스포머
text->au 트랜스포머
트랜스포머
1531/1531 [==============================] - ETA: 0s - loss: 1.4993 - accuracy: 0.3927 - precision: 0.6901 - recall: 0.2289 - f1score: 0.3170t
트랜스포머
text->au 트랜스포머
트랜스포머
1531/1531 [==============================] - 52s 28ms/step - loss: 1.4990 - accuracy: 0.3928 - precision: 0.6902 - recall: 0.2290 - f1score: 0.3171 - val_loss: 1.3315 - val_accuracy: 0.8318 - val_precision: 0.8281 - val_recall: 0.8281 - val_f1score: 0.8281
Epoch 2/5
1531/1531 [==============================] - 43s 28ms/step - loss: 0.4310 - accuracy: 0.8527 - precision: 0.8688 - recall: 0.8293 - f1score: 0.8477 - val_loss: 1.6606 - val_accuracy: 0.8223 - val_precision: 0.8193 - val_recall: 0.8180 - val_f1score: 0.8186
Epoch 3/5
1531/1531 [==============================] - 41s 27ms/step - loss: 0.2514 - accuracy: 0.9351 - precision: 0.9362 - recall: 0.9321 - f1score: 0.9341 - val_loss: 1.6825 - val_accuracy: 0.8318 - val_precision: 0.8281 - val_recall: 

In [56]:
_loss, _acc, _precision, _recall, _f1score = model.evaluate([[np.array(x_test['mfcc_scaled'].to_list()),np.array(x_test['text_tokenize'].to_list()),np.array(x_test['temp+eda'].to_list())]], y_test, batch_size=16, verbose=1)
print('loss: {:.3f}, accuracy: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1score: {:.3f}'.format(_loss, _acc, _precision, _recall, _f1score))

161/161 [==============================] - 2s 12ms/step - loss: 1.9388 - accuracy: 0.8029 - precision: 0.8036 - recall: 0.8036 - f1score: 0.8036
loss: 1.939, accuracy: 0.803, precision: 0.804, recall: 0.804, f1score: 0.804
